In [1]:
%load_ext lab_black

In [2]:
import pathlib
import itertools
import datetime
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import networkx as nx

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    OneHotEncoder,
    PolynomialFeatures,
    power_transform,
    PowerTransformer,
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin
from preprocessing import numeric_object_split, z_filter
from IPython.display import clear_output

The purpose of this notebook is to aggregate data on multiple different linear models. As interesting as it is to theorize the effects of feature engineering, scaling, imputing, etc. during EDA, ultimately what determines what the best actions are going to be is determined by model performance. Thus, this notebook will declare all options I wish to explore, iterate through all combinations of them, create a model with the corresponding hyperparameters, fit the data and return the $R^2$ and RMSE values. The parameters and scores are then saved to a file for use in model tuning. 

## Model Options

The first option I wish to explore is imputation method. The first being a simple imputation of the mean value of the feature, and the second being a K-Nearest Neighbors imputation.

In [3]:
IMPUTER_VARIANTS = {
    "mean": SimpleImputer(strategy="mean"),
    "knn": KNNImputer(),
}

Next, there are the model versions. The first is the standard variant of linear regression, which is ordinary least squares. The next two are Lasso and Ridge, which include an $L_1$ and $L_2$ regularization penalty, respectively. Finally, ElasticNet models will be tested, which include both an $L_1$ *and* $L_2$ penalty. It is the ElasticNet model that I assume will form the best predictions. 

In [4]:
LAMBDAS = [0.01] + list(np.linspace(0, 0.1, 5)[1:])

REGRESSOR_VARIANTS = (
    {"ols": LinearRegression()}
    | {f"lasso_{l1}": Lasso(alpha=l1) for l1 in LAMBDAS}
    | {f"ridge_{l2}": Lasso(alpha=l2) for l2 in LAMBDAS}
    | {
        f"elasticnet_{l1}_{l2}": ElasticNet(alpha=l1 + l2, l1_ratio=l1 / (l1 + l2))
        for l1, l2 in itertools.product(LAMBDAS, LAMBDAS)
    }
)

The two versions of the target variable are available to fit, being the provided target of sale_price, and my constructed alternative target price_per_area

In [5]:
TARGETS = ["saleprice", "price_per_area"]

Next, there are three different sets of features which I explored during EDA. The first is whether to use all features, or only those with numeric data types. While having more features to learn would in theory grant a better fit, it can also lead to overfitting, and thus limiting the model to only numerical data may increase performance on validation data. 

For area-like features, it was noted that gr_liv_area was the sum of the areas of the first and second floors. Whether to include total livable area or the independent floor 1 and 2 areas is something that the model will determine. There are three versions of the area set: total livable area only, floor 1 and 2 area only, and all included.

For time, a similar approach was taken. The sale time was originally encoded in two columns, corresponding to the year and month. Thus one variant of the areas leaves uses these two columns and one-hot encodes them. The next uses a combined year-month string which is encoded. A third variant converts the date to a timestamp and fits on it. The fourth variant uses both the timestamp and year-month columns. Finally, a fifth variant uses all of the time columns.

In [6]:
ames_train = pd.read_csv(
    "../datasets/train_preprocessed.csv",
    dtype={"ms_subclass": object, "yr_sold": object, "mo_sold": object},
)

numerics, objects = numeric_object_split(ames_train)

FEATURE_SETS = {
    # name: columns to drop
    "full": [],
    "numeric": objects.columns,
}
AREA_SETS = {
    "full": [],
    "total_grade_area": ["1st_flr_sf", "2nd_flr_sf"],
    "separate_floors": ["gr_liv_area"],
}
TIME_SETS = {
    "full": [],
    "timestamp": ["year_month", "yr_sold", "mo_sold"],
    "year_month": ["sale_timestamp", "yr_sold", "mo_sold"],
    "year_and_month": ["sale_timestamp", "year_month"],
    "timestamp_and_year_month": ["yr_sold", "mo_sold"],
}

One of the things I will attempt to implement in this model is data splitting: breaking the data into logical subsets, and fitting each subset with its own regression model in a matter similar to a regression tree. EDA of the training data made two logical splits apparent to me. The first was for homes which had a pid starting with "5" or "9", as the groups were fairly even. The second was to split on if a home had an "average" overall condition (5). Homes with conditions *other* than 5 seemed to have their price increase linearly with the variable, but there were so many homes with an average condition that the overall simple fit was incredibly weak. As such, it made sense to me to analyze average homes and non-average homes as two separate groups. These two splitting methods define four split sets: neither, pid only, condition only, and both. Additionally, a function is declared here that will construct a set of indices to be included in each split so that the data can be easily decomposed into subsets.

In [7]:
SPLIT_SET = {
    # name: (column, value) to split on
    "no_split": [],
    "pid_split": [("pid_5", 1)],
    "average_condition_split": [("overall_cond", 5)],
    "pid_condition_split": [("pid_5", 1), ("overall_cond", 5)],
}


def splitter(split_index, size):
    # given a number of splits to perform
    # this function will return a list of
    # boolean indexers to apply to
    # the data
    # if split index is empty, returns a single full split
    # in general returns 2 ** num_splitters splits
    num_splitters = len(split_index)

    num_arrays = 2**num_splitters
    truth_arrays = np.array([[True for _ in range(size)] for n in range(num_arrays)])

    binary = [True, False]

    for i, combo in enumerate(itertools.product(binary, repeat=num_splitters)):
        for j, boolean in enumerate(combo):
            if boolean:
                truth_arrays[i] *= split_index[j]
            else:
                truth_arrays[i] *= np.logical_not(split_index[j])

    return truth_arrays

Finally, we define a few thresholds for the data. The z threshold is the maximum value of the absolute value of the z-scored data to be included for fitting. This is to prevent outliers from having too much of an effect on the regression process. The covariance threshold is the maximum amount of the absolute value of covariance any two standardized features are allowed to share. This is a systematic way to remove colinear variables from the dataset prior to regression. Both z threshold and covariance threshold were only given a single value to reduce the analysis time, as this process is already quite lengthy.

Target covariance threshold represents the minimum amount of correlation a feature must have with the target. Larger values will filter out the weaker features, leaving only a few well-fitting columns to be used in regression. This will help to reduce overfitting.

In [8]:
COVARIANCE_THRESHOLD = 0.9
TARGET_COVARIANCE_THRESHOLDS = [0.0, 0.2, 0.4]
Z_THRESHOLD = 2.5

Here, the total number of iterations is calculated (51840) to give an estimate of progress during the grid search. As my machine is fairly powerful, this will take only a few hours. 

In [9]:
tot_iterations = (
    len(IMPUTER_VARIANTS)
    * len(TARGET_COVARIANCE_THRESHOLDS)
    * len(TARGETS)
    * len(SPLIT_SET)
    * len(FEATURE_SETS)
    * len(TIME_SETS)
    * len(AREA_SETS)
    * len(REGRESSOR_VARIANTS)
)

Additionally, I declare two functions to be used during preprocessing. The first is rather comical, and exists to *reintroduce* null values to a dataset. This is due to the way that sklearn handles null values during one-hot encoding, which is to create their own feature. I didn't like this, and thought that examples with null values should be left null for a proper imputer to handle, but this doesn't seem to be an option sklearn's OneHotEncoder provides. Therefore, I chose instead to provide the encoder with a list of categories to explicitly create features, then checked if *all* relevant columns for the original feature were zero. If they were, the zeros were replaced with np.nan. The reason I didn't impute *before* encoding is so that I could make use of a KNN imputer, which requires properly-scaled, numeric features. Thus the order of operations was forced to be scale -> encode -> put nulls back -> impute.

The second function is to check if, after other filtering methods have been applied, any features are left with only a single value. As these features now have zero variance, this can cause issues during computation and must be dropped.

In [10]:
def renanner(X, categories):
    # reintroduce nans to the dataset (lol)
    col = 0
    for cat in categories:
        end_col = col + len(cat)
        subset = X[:, col:end_col]
        for n, row in enumerate(subset):
            if np.all(row == 0):
                new_row = np.array([np.nan for i in range(len(row))])
                X[n, col:end_col] = new_row

    return X


def homogenous_columns(X):
    unique_counts = np.array(
        [len(u) for u in [np.unique(X[:, col]) for col in range(X.shape[1])]]
    )
    return unique_counts == 1

# Model Selection

The following code cell is large and dense, so I will outline its logic here in addition to comments. For each step in the loop below:

1. The relevant features are selected, scaled, encoded, and have their missing values imputed.

2. Features with high bivariate correlation and/or low target correlation are systematically dropped. Which features are dropped are *not* tracked here, but will be in the model tuning phase.

3. Features are transformed via a Yeo-Johnson transformation. The transformation parameter is not tracked.

4. Data is split into subsets and fit with a linear model. Polynomial features are not tested here due to memory constraints.

5. Model parameters and scores are written to a file

The "best" model as determined by cross-validation score is continuously displayed as the output of the cell, as well as a few progress metrics.

In [11]:
# a csv file is created to store model hyperparameters and scores
with open(
    f"../etc/model_results_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.csv", "w+"
) as log:
    log.write(
        "regressor,imputer,feature_set,area_set,time_set,split_name,covariance_threshold,target,train_score,val_score,cross_val_score,train_rmse,val_rmse\n"
    )
    # training data is z filtered
    ames_train = z_filter(ames_train, high=Z_THRESHOLD, low=-Z_THRESHOLD)

    # a few dummy values are declared for metrics I will be tracking
    start_time = time.time()
    iterations_completed = 0
    best_cross_val_score = 0
    best_rmse = 0
    best_config = [0, 1, 2, 3, 4, 5, 6, 7]

    # feature sets are looped through
    for feature_set_name, feature_set in FEATURE_SETS.items():

        # time sets are looped through
        for time_set_name, time_set in TIME_SETS.items():

            # area sets are looped through
            for area_set_name, area_set in AREA_SETS.items():

                # target variants are looped through
                for target in TARGETS:
                    # target values are declared and manually standardized
                    # the mean and standard dev are stored for destanadardizing
                    # this will ensure RMSE is interpretable
                    y_full = ames_train[target]
                    y_mean = y_full.mean()
                    y_std = y_full.std()
                    y = np.array((y_full - y_mean) / y_std)

                    # features are dropped based on the feature, time, and area set
                    features_to_drop = list(
                        np.unique(np.concatenate([feature_set, time_set, area_set]))
                    )

                    # a "transformer" is defined to transform the target to sale price
                    # if target is sale price, the tranformer is identity
                    # otherwise, it is surface area
                    x_transform = np.ones(y.shape[0])
                    if target == "price_per_area":
                        features_to_drop.append("gr_liv_area")
                        features_to_drop.append("1st_flr_sf")
                        features_to_drop.append("2nd_flr_sf")

                        x_transform = np.array(ames_train["gr_liv_area"])

                    # the full array of features is formally decalered here
                    X_full = ames_train[
                        [
                            col
                            for col in ames_train.columns
                            if col not in features_to_drop and col not in TARGETS
                        ]
                    ]

                    # it is easiest to define split conditions
                    # on unscaled data so the split mask arrays
                    # are created here
                    # the data is not yet split
                    split_indices = {}
                    for name, split in SPLIT_SET.items():
                        split_indices[name] = []
                        for col, value in split:
                            split_indices[name].append(X_full[col] == value)

                    # the dataframe is broken into numeric and categorical subsets
                    X_numerics, X_objects = numeric_object_split(X_full)

                    # numeric values are scaled
                    scaler = StandardScaler()
                    scaled = scaler.fit_transform(X_numerics)

                    # expected categories are declared
                    categories = [X_objects[col].unique() for col in X_objects.columns]
                    categories = [cat[[c != "nan" for c in cat]] for cat in categories]

                    # categorical features are one-hot encoded
                    ohe = OneHotEncoder(categories=categories, sparse=False)
                    encoded = ohe.fit_transform(X_objects)
                    # null values are reintroduced for proper imputation
                    encoded = renanner(encoded, categories)

                    # the formatted numeric and object columns are recombined
                    X_full = np.concatenate([scaled, encoded], axis=1)

                    # imputer variants are looped through
                    for imputer_name, imputer in IMPUTER_VARIANTS.items():
                        # null values are imputed
                        imputed = imputer.fit_transform(X_full)

                        # drop columns with only a single value
                        imputed = imputed[:, ~homogenous_columns(imputed)]

                        # this part's cool
                        # bivariate correlations are computed for all features
                        # each feature is treated as a node on a graph
                        # features with strong colinearity have an edge drawn between their nodes
                        corr = np.corrcoef(imputed, rowvar=False)
                        correlation_graph = nx.Graph()
                        correlation_graph.add_nodes_from(range(corr.shape[0]))
                        for row in range(corr.shape[0]):
                            high_bivariate_corr = np.where(
                                np.abs(corr[row]) >= COVARIANCE_THRESHOLD
                            )[0]
                            for idx in high_bivariate_corr:
                                correlation_graph.add_edge(row, idx)
                        correlation_graph.remove_edges_from(
                            nx.selfloop_edges(correlation_graph)
                        )

                        # once the graph is completed, nodes with edges are randomly removed
                        # the node is drawn from the set of nodes with the highest number of edges
                        # this process is repeated until the graph is unconnected
                        # this leaves only nodes with *acceptable* levels of colinearity
                        # the remaining nodes are then used to index the full dataset
                        max_degree = 1
                        while max_degree > 0:
                            degree = correlation_graph.degree()
                            max_degree = np.max([d for n, d in degree])
                            worst_nodes = [n for n, d in degree if d == max_degree]
                            correlation_graph.remove_node(np.random.choice(worst_nodes))

                        features_to_keep = correlation_graph.nodes

                        X_no_bivariate_corr = imputed[:, features_to_keep]

                        # target covariance thresholds are iterated over
                        # we note here that because the data is standardized
                        # cov and corr are interchangable values
                        for target_covariance_threshold in TARGET_COVARIANCE_THRESHOLDS:
                            # feature target correlations are calculated
                            target_corr = np.array(
                                [
                                    np.corrcoef(y_full, x)[0, 1]
                                    for x in X_no_bivariate_corr.T
                                ]
                            )

                            # low correlation features are dropped
                            features_to_keep = np.where(
                                np.abs(target_corr) >= target_covariance_threshold
                            )[0]

                            # a yeo-johnson transform is applied to the features
                            X = power_transform(
                                X_no_bivariate_corr[:, features_to_keep]
                            )

                            # regressor versions are looped through
                            for regressor_name, regressor in REGRESSOR_VARIANTS.items():

                                # split methods are looped through
                                for split_name, split_index in split_indices.items():

                                    # empty variables are declared to track
                                    # which examples go into which split
                                    # so that the data can be properly reconstructed
                                    y_preds = np.zeros(y.shape[0])

                                    indices = np.arange(y.shape[0])

                                    train_indices, val_indices = np.array(
                                        [], dtype=int
                                    ), np.array([], dtype=int)

                                    # cross val score is taken as a weighted average
                                    # of cross val scores of individual splits
                                    cross_scores = []
                                    split_sizes = []

                                    # split indexing arrays are declared
                                    splits = splitter(split_index, X.shape[0])

                                    # data subsets are looped through
                                    for split in splits:

                                        # train test splits are created
                                        (
                                            X_train,
                                            X_val,
                                            y_train,
                                            y_val,
                                            indices_train,
                                            indices_val,
                                        ) = train_test_split(
                                            X[split],
                                            y[split],
                                            indices[split],
                                            train_size=0.7,
                                            random_state=42,
                                        )

                                        # example indices are tracked
                                        (
                                            indices_train,
                                            indices_val,
                                        ) = indices_train.astype(
                                            int
                                        ), indices_val.astype(
                                            int
                                        )

                                        train_indices = np.append(
                                            train_indices, indices_train
                                        )
                                        val_indices = np.append(
                                            val_indices, indices_val
                                        )

                                        # the model is fit to the train split
                                        model = regressor.fit(X_train, y_train)

                                        # randomized folds are created
                                        folds = KFold(shuffle=True, random_state=42)

                                        # the cross score for the split is calculated
                                        cross_scores.append(
                                            np.mean(
                                                cross_val_score(
                                                    model, X[split], y[split], cv=folds
                                                )
                                            )
                                        )

                                        split_sizes.append(len(X[split]))

                                        # predictions for the split are made
                                        y_pred_train = model.predict(X_train)
                                        y_pred_val = model.predict(X_val)

                                        y_preds[indices_train] += y_pred_train
                                        y_preds[indices_val] += y_pred_val

                                    # y value is detransformed to get back to sale price
                                    # with units of dollars
                                    y_preds_descaled = (
                                        y_preds * y_std + y_mean
                                    ) * x_transform
                                    y_true_descaled = (y * y_std + y_mean) * x_transform

                                    # relevant metrics are ccalculated
                                    train_score = r2_score(
                                        y_preds_descaled[train_indices],
                                        y_true_descaled[train_indices],
                                    )
                                    train_rmse = mean_squared_error(
                                        y_preds_descaled[train_indices],
                                        y_true_descaled[train_indices],
                                        squared=False,
                                    )

                                    val_score = r2_score(
                                        y_preds_descaled[val_indices],
                                        y_true_descaled[val_indices],
                                    )
                                    val_rmse = mean_squared_error(
                                        y_preds_descaled[val_indices],
                                        y_true_descaled[val_indices],
                                        squared=False,
                                    )

                                    cross_score = np.average(
                                        cross_scores, weights=split_sizes
                                    )

                                    # information is formatted for file writing
                                    config = [
                                        regressor_name,
                                        imputer_name,
                                        feature_set_name,
                                        area_set_name,
                                        time_set_name,
                                        split_name,
                                        target_covariance_threshold,
                                        target,
                                    ]

                                    scores = [
                                        train_score,
                                        val_score,
                                        cross_score,
                                        train_rmse,
                                        val_rmse,
                                    ]

                                    # the best calculated cross val score is tracked for printing
                                    # along with the rmse and config associated with that score
                                    if cross_score > best_cross_val_score:
                                        best_cross_val_score = cross_score
                                        best_rmse = val_rmse
                                        best_config = config

                                    # lots of time and progress stuff is calculated for a nice printout
                                    elapsed_time = time.time() - start_time

                                    elapsed_time_tuple = str(
                                        datetime.timedelta(seconds=elapsed_time)
                                    ).split(":")

                                    elapsed_time_string = f"{elapsed_time_tuple[0]}:{elapsed_time_tuple[1]}:{round(float(elapsed_time_tuple[2])):02}"

                                    iterations_completed += 1

                                    estimated_time_remaining = (
                                        elapsed_time
                                        * tot_iterations
                                        / iterations_completed
                                    ) - elapsed_time

                                    estimated_remaining_time_tuple = str(
                                        datetime.timedelta(
                                            seconds=estimated_time_remaining
                                        )
                                    ).split(":")

                                    estimated_remaining_time_string = f"{estimated_remaining_time_tuple[0]}:{estimated_remaining_time_tuple[1]}:{round(float(estimated_remaining_time_tuple[2])):02}"

                                    to_print = (
                                        f"Iterations completed: {iterations_completed}/{tot_iterations}\n"
                                        f"Elapsed time: {elapsed_time_string}\n"
                                        f"Estimated time remaining: {estimated_remaining_time_string}\n\n"
                                        f"Best cross-validation score: {round(best_cross_val_score, 5)}\n"
                                        f"Best RMSE: {round(best_rmse, 3)}\n\n"
                                        f"Regressor: {best_config[0]}\n"
                                        f"Imputer: {best_config[1]}\n"
                                        f"Feature set: {best_config[2]}\n"
                                        f"Area set: {best_config[3]}\n"
                                        f"Time set: {best_config[4]}\n"
                                        f"Split: {best_config[5]}\n"
                                        f"Covariance threshold: {best_config[6]}\n"
                                        f"Target: {best_config[7]}"
                                    )

                                    logline = (
                                        ",".join([str(s) for s in config + scores])
                                        + "\n"
                                    )
                                    log.write(logline)

                                    clear_output(wait=True)
                                    print(to_print)

Iterations completed: 51840/51840
Elapsed time: 0:58:27
Estimated time remaining: 0:00:00

Best cross-validation score: 0.91338
Best RMSE: 16859.357

Regressor: lasso_0.01
Imputer: mean
Feature set: full
Area set: full
Time set: full
Split: no_split
Covariance threshold: 0.0
Target: saleprice


And with that, we have sufficient data to draw conclusions on how to design a proper model. 

# Model Tuning

In [12]:
model_stats = pd.read_csv("../assets/model_results_20221208170253.csv")
model_stats.sort_values(by=["val_rmse"], ascending=True).head(25)

,regressor,imputer,feature_set,area_set,time_set,split_name,covariance_threshold,target,train_score,val_score,cross_val_score,train_rmse,val_rmse
1732,lasso_0.01,mean,full,total_grade_area,full,no_split,0.0,saleprice,0.921374,0.905949,0.913029,14925.817030,16883.407446
1752,ridge_0.01,mean,full,total_grade_area,full,no_split,0.0,saleprice,0.921374,0.905949,0.913029,14925.817030,16883.407446
436,lasso_0.01,knn,full,full,full,no_split,0.0,saleprice,0.921177,0.905692,0.913085,14954.431728,16884.491988
456,ridge_0.01,knn,full,full,full,no_split,0.0,saleprice,0.921177,0.905692,0.913085,14954.431728,16884.491988
17304,ridge_0.01,mean,full,total_grade_area,year_and_month,no_split,0.0,saleprice,0.918833,0.905774,0.912698,15155.059352,16888.122382
17284,lasso_0.01,mean,full,total_grade_area,year_and_month,no_split,0.0,saleprice,0.918833,0.905774,0.912698,15155.059352,16888.122382
2184,ridge_0.01,knn,full,total_grade_area,full,no_split,0.0,saleprice,0.921494,0.905650,0.912629,14917.629433,16912.340991
2164,lasso_0.01,knn,full,total_grade_area,full,no_split,0.0,saleprice,0.921494,0.905650,0.912629,14917.629433,16912.340991
22488,ridge_0.01,mean,full,total_grade_area,timestamp_and_year_month,no_split,0.0,saleprice,0.920925,0.905442,0.912633,14965.535683,16915.209837
22468,lasso_0.01,mean,full,total_grade_area,timestamp_and_year_month,no_split,0.0,saleprice,0.920925,0.905442,0.912633,14965.535683,16915.209837


The output above allows me to draw a few conclusions about what the final version of the model should look like.

1. Lasso, Ridge, and ElasticNet all perform comparably, and they all share the smallest value of the regularization penalty. As such, the final model will be constructed as an ElasticNet, as it is the more general of the three. Whether or not $\lambda = 0.01$ is the correct value to use will be determined below.

2. Imputing with the mean and KNN produce nearly identical results, with mean yielding slightly better RMSE values and KNN yielding slightly better cross-validation $R^2$. This difference may be an error in how the calculation of scores was handled, but it is not sufficiently large to be reason to worry. The final model will impute missing values with the mean. 

3. All of the features should be used, with the execption of dropping the 1st and 2nd floor areas.

4. Data splitting seems to be a no-go. The exact effects of data splitting will be discussed in the results notebook, but it is clear that the most successful models did not utilize it.

5. No features were filtered by target covariance. Everything that made it through other filters was included. 

6. Saleprice proved to be the better target to predict in the best cases.

Now, the final model will be initialized and tuned for the ideal $\lambda$ values.

In [13]:
feature_set = FEATURE_SETS["full"]
time_set = TIME_SETS["full"]
area_set = AREA_SETS["total_grade_area"]
target = "saleprice"
imputer = IMPUTER_VARIANTS["mean"]

LAMBDAS_FINE = np.linspace(0.0, 0.02, 41)

In [14]:
ames_train = pd.read_csv(
    "../datasets/train_preprocessed.csv",
    dtype={"ms_subclass": object, "yr_sold": object, "mo_sold": object},
)
ames_train = ames_train.reindex(sorted(ames_train.columns), axis=1)
ames_train = z_filter(ames_train, high=Z_THRESHOLD, low=-Z_THRESHOLD)
train_id = ames_train["id"]
ames_train = ames_train.drop(columns=["id"])

ames_test = pd.read_csv(
    "../datasets/test_preprocessed.csv",
    dtype={"ms_subclass": object, "yr_sold": object, "mo_sold": object},
)
ames_test = ames_test.reindex(sorted(ames_test.columns), axis=1)
test_id = ames_test["id"]
ames_test = ames_test.drop(columns=["id"])

In [15]:
y_training = ames_train[target]

y_mean = y_training.mean()
y_std = y_training.std()

y_training = (y_training - y_mean) / y_std

In [16]:
features_to_drop = list(np.unique(np.concatenate([feature_set, time_set, area_set])))

relevant_columns = [
    col
    for col in ames_train.columns
    if col not in features_to_drop and col not in TARGETS
]

X_training = ames_train.drop(columns=TARGETS)

X_training = ames_train[relevant_columns]
X_testing = ames_test[relevant_columns]

# the dataframe is broken into numeric and categorical subsets
X_numerics_training, X_objects_training = numeric_object_split(X_training)
X_numerics_testing, X_objects_testing = numeric_object_split(X_testing)


# numeric values are scaled
scaler = StandardScaler()
training_scaled = scaler.fit_transform(X_numerics_training)
testing_scaled = scaler.transform(X_numerics_testing)


# expected categories are declared
categories = [X_objects_training[col].unique() for col in X_objects_training.columns]
categories = [cat[[c != "nan" for c in cat]] for cat in categories]

# categorical features are one-hot encoded
ohe = OneHotEncoder(categories=categories, sparse=False, handle_unknown="ignore")
training_encoded = ohe.fit_transform(X_objects_training)
testing_encoded = ohe.transform(X_objects_testing)


remaining_features = np.concatenate(
    [scaler.get_feature_names_out(), ohe.get_feature_names_out()]
)

# null values are reintroduced for proper imputation
training_encoded = renanner(training_encoded, categories)
testing_encoded = renanner(testing_encoded, categories)

# the formatted numeric and object columns are recombined
X_training = np.concatenate([training_scaled, training_encoded], axis=1)
X_testing = np.concatenate([testing_scaled, testing_encoded], axis=1)

In [17]:
training_imputed = imputer.fit_transform(X_training)
testing_imputed = imputer.transform(X_testing)

drop_mask = ~homogenous_columns(training_imputed)
columns_to_drop = np.array([n for n, t in enumerate(drop_mask) if not t])
training_imputed = training_imputed[:, drop_mask]
testing_imputed = testing_imputed[:, drop_mask]
remaining_features = remaining_features[drop_mask]

In [18]:
# this part's cool
# bivariate correlations are computed for all features
# each feature is treated as a node on a graph
# features with strong colinearity have an edge drawn between their nodes
corr = np.corrcoef(training_imputed, rowvar=False)
correlation_graph = nx.Graph()
correlation_graph.add_nodes_from(range(corr.shape[0]))
for row in range(corr.shape[0]):
    high_bivariate_corr = np.where(np.abs(corr[row]) >= COVARIANCE_THRESHOLD)[0]
    for idx in high_bivariate_corr:
        correlation_graph.add_edge(row, idx)
correlation_graph.remove_edges_from(nx.selfloop_edges(correlation_graph))

# once the graph is completed, nodes with edges are randomly removed
# the node is drawn from the set of nodes with the highest number of edges
# this process is repeated until the graph is unconnected
# this leaves only nodes with *acceptable* levels of colinearity
# the remaining nodes are then used to index the full dataset
max_degree = 1
while max_degree > 0:
    degree = correlation_graph.degree()
    max_degree = np.max([d for n, d in degree])
    worst_nodes = [n for n, d in degree if d == max_degree]
    correlation_graph.remove_node(np.random.choice(worst_nodes))

features_to_keep = correlation_graph.nodes

X_training = training_imputed[:, features_to_keep]
X_testing = testing_imputed[:, features_to_keep]
remaining_features = remaining_features[features_to_keep]

In [19]:
# a yeo-johnson transform is applied to the features
yjt = PowerTransformer()
X_training = yjt.fit_transform(X_training)
X_testing = yjt.fit_transform(X_testing)
yjt_parameters = yjt.lambdas_

/home/awalsh/.venvs/ga/lib64/python3.11/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/awalsh/.venvs/ga/lib64/python3.11/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/awalsh/.venvs/ga/lib64/python3.11/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/awalsh/.venvs/ga/lib64/python3.11/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [20]:
num_its = len(LAMBDAS_FINE) ** 2

best_score = 0
best_l1 = 0
best_l2 = 0
i = 0
for l1 in LAMBDAS_FINE:
    for l2 in LAMBDAS_FINE:
        en = ElasticNet(alpha=l1 + l2, l1_ratio=l1 / (l1 + l2))
        folds = KFold(shuffle=True, random_state=42)
        try:
            score = np.mean(cross_val_score(en, X_training, y_training, cv=folds))
        except:
            score = -1
        if score > best_score:
            best_score = score
            best_l1 = l1
            best_l2 = l2
        clear_output(wait=True)
        i += 1
        print(f"{i}/{num_its}", best_l1, best_l2, best_score)

1681/1681 0.005 0.0 0.9146916958974453


According to the above, the best model is a Lasso regression with $\lambda=0.005$. Of course, now that we've eliminated the $L_2$ penalty, a hyperfine grid search can't hurt.

In [21]:
LAMBDAS_HYPERFINE = np.linspace(0.0, 0.01, 1001)

best_score = 0
best_l1 = 0
i = 0
for l1 in LAMBDAS_HYPERFINE:
    la = Lasso(alpha=l1)
    folds = KFold(shuffle=True, random_state=42)
    try:
        score = np.mean(cross_val_score(la, X_training, y_training, cv=folds))
    except:
        score = -1
    if score > best_score:
        best_score = score
        best_l1 = l1
    clear_output(wait=True)
    i += 1
    print(f"{i}/{len(LAMBDAS_HYPERFINE)}", best_l1, best_score)

1001/1001 0.0048000000000000004 0.9146973415997225


The final model used in this analysis will be a Lasso regression with $\lambda=0.00497$.

In [25]:
la = Lasso(alpha=0.00497)

model = la.fit(X_training, y_training)
domain_pred = la.predict(X_training)

y_pred = model.predict(X_testing)

betas = model.coef_

y_training_descaled = y_training * y_std + y_mean
domain_pred_descaled = domain_pred * y_std + y_mean
y_pred_descaled = y_pred * y_std + y_mean

with open("../assets/domain_predictions.csv", "w+") as pred_file:
    header = "y_pred,y_true\n"
    pred_file.write(header)
    for y_p, y_t in zip(domain_pred_descaled, y_training_descaled):
        line = f"{y_p},{y_t}\n"
        pred_file.write(line)

with open("../assets/coefs.csv", "w+") as coef_file:
    header = "feature,beta,lambda\n"
    coef_file.write(header)
    for n, feature in enumerate(remaining_features):
        line = f"{feature},{betas[n]},{yjt_parameters[n]}\n"
        coef_file.write(line)


with open("../datasets/submission.csv", "w+") as kaggle:
    header = "Id,SalePrice\n"
    kaggle.write(header)
    for n, pred in enumerate(y_pred_descaled):
        line = f"{test_id[n]},{pred}\n"
        kaggle.write(line)